In [1]:
# append the module root to Python path
import sys

sys.path.append("../src")

In [2]:
from mygo.datasets import MCTSDataset
from mygo.model import TinyModel
from torch.utils.data import DataLoader
import torch
from torch import nn

In [3]:
torch.manual_seed(25565)  # for reproducibility

In [4]:
def transform(xs):
    """Transform H-W input to C-H-W format, where C = 1."""
    xs = torch.from_numpy(xs)
    # return xs.reshape(1, *xs.shape)
    return xs.unsqueeze(0)

In [5]:
train_data = MCTSDataset(
    root="../data",
    train=True,
    transform=transform,
)

test_data = MCTSDataset(
    root="../data",
    train=False,
    transform=transform,
)

In [6]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    batches = len(dataloader)
    model.train()
    for i, (xs, ys) in enumerate(dataloader):
        optimizer.zero_grad()

        pred = model(xs)
        loss = loss_fn(pred, ys)

        loss.backward()
        optimizer.step()

        if i % 10 == 9:
            print(f"loss: {loss.item():>7f} [{i + 1:>2d}/{batches:>2d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0.0

    with torch.no_grad():
        for xs, ys in dataloader:
            pred = model(xs)
            test_loss += loss_fn(pred, ys).item()
            correct += (pred.argmax(1) == ys).type(torch.float).sum().item()

    test_loss /= batches
    accuracy = correct / size * 100

    print(f"\nTest Accuracy: {accuracy:>.1f}%\nAvg Loss: {test_loss:>7f}\n")

In [8]:
model = TinyModel(9)
print(model)

total_params = sum(p.numel() for p in model.parameters())
print(f"parameters: {total_params:,}")

TinyModel(
  (conv_stack): Sequential(
    (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=81, bias=True)
    (4): Softmax(dim=1)
  )
)
parameters: 456,545


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters())
epoches = 100

for i in range(epoches):
    print(f"Epoch {i + 1}\n{'-' * 25}")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)

Epoch 1
-------------------------
loss: 4.394418 [10/39]
loss: 4.394964 [20/39]
loss: 4.394269 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.394490

Epoch 2
-------------------------
loss: 4.394292 [10/39]
loss: 4.394801 [20/39]
loss: 4.394701 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.394513

Epoch 3
-------------------------
loss: 4.394733 [10/39]
loss: 4.394656 [20/39]
loss: 4.394507 [30/39]

Test Accuracy: 0.0%
Avg Loss: 4.394607

Epoch 4
-------------------------
loss: 4.393892 [10/39]
loss: 4.394433 [20/39]
loss: 4.394438 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.394583

Epoch 5
-------------------------
loss: 4.394439 [10/39]
loss: 4.394459 [20/39]
loss: 4.393935 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.394433

Epoch 6
-------------------------
loss: 4.394343 [10/39]
loss: 4.394471 [20/39]
loss: 4.394193 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.394689

Epoch 7
-------------------------
loss: 4.394497 [10/39]
loss: 4.394354 [20/39]
loss: 4.394849 [30/39]

Test Accuracy: 1.5%
Avg Loss: 4.

In [10]:
test_loop(train_loader, model, loss_fn)


Test Accuracy: 13.6%
Avg Loss: 4.286269



In [11]:
torch.save(model.state_dict(), "../models/tiny_weights.pt")